In [1]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import DataLoader

# GPU OR CPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU")
else:
    device = torch.device("cpu")
    print("CPU")


batch_size = 1000
num_epoch = 50

GPU


In [2]:
train_dataset = torchvision.datasets.LFWPeople(root='./dataset/',
                                           split="train",
                                           transform=transforms.Compose([
                                               transforms.Resize((28,28)),
                                               transforms.Grayscale(),
                                               transforms.ToTensor()]),
                                           download=True)

test_dataset = torchvision.datasets.LFWPeople(root='./dataset/',
                                           split="test",
                                           transform=transforms.Compose([
                                              transforms.Resize((28,28)),
                                               transforms.Grayscale(),
                                               transforms.ToTensor()]),
                                           download=True)

train_dataset = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)



  0%|          | 0/243346528 [00:00<?, ?it/s]

Extracting ./dataset/lfw-py/lfw-funneled.tgz to ./dataset/lfw-py


  0%|          | 0/66403 [00:00<?, ?it/s]

  0%|          | 0/94727 [00:00<?, ?it/s]

Using downloaded and verified file: ./dataset/lfw-py/lfw-funneled.tgz
Extracting ./dataset/lfw-py/lfw-funneled.tgz to ./dataset/lfw-py


  0%|          | 0/28334 [00:00<?, ?it/s]

Using downloaded and verified file: ./dataset/lfw-py/lfw-names.txt


In [3]:
for data, label in train_dataset:
    print(data , label)
    print("--------")

tensor([[[[0.0275, 0.0941, 0.1490,  ..., 0.4784, 0.4314, 0.0706],
          [0.0510, 0.1059, 0.1804,  ..., 0.5373, 0.4706, 0.0784],
          [0.0588, 0.1176, 0.1922,  ..., 0.5529, 0.5059, 0.1137],
          ...,
          [0.1020, 0.1882, 0.4510,  ..., 0.8392, 0.8549, 0.8353],
          [0.1333, 0.1412, 0.2941,  ..., 0.8275, 0.8275, 0.8275],
          [0.1451, 0.1569, 0.1804,  ..., 0.8157, 0.8078, 0.8196]]],


        [[[0.0118, 0.0510, 0.0510,  ..., 0.0588, 0.0588, 0.0706],
          [0.0196, 0.0745, 0.0784,  ..., 0.0824, 0.0863, 0.0941],
          [0.0235, 0.0941, 0.1059,  ..., 0.1176, 0.1176, 0.1216],
          ...,
          [0.0706, 0.4157, 0.3804,  ..., 0.2235, 0.2431, 0.2196],
          [0.0235, 0.1451, 0.1412,  ..., 0.1490, 0.1765, 0.1686],
          [0.0196, 0.1137, 0.1216,  ..., 0.1176, 0.1529, 0.1333]]],


        [[[0.0980, 0.1098, 0.1098,  ..., 0.1020, 0.0824, 0.0078],
          [0.2431, 0.2824, 0.2824,  ..., 0.2667, 0.2235, 0.0235],
          [0.2706, 0.3216, 0.3176,  ..

In [4]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()

        # input: 28 * 28 * 1
        # filter: 3 * 3 * 1 * 16
        # padding: 32 * 32 * 1
        # out1: 32 - (3 - 1) => 30 * 30 * 16
        #pooling: 15 * 15 * 16
        self.layer1 = nn.Sequential(               # filter = 3 * 3 * 16
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1,padding=2),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))

        # input: 15 * 15 * 16
        # filter: 3 * 3 * 16 * 32
        # padding: 19 * 19 * 1
        # out2:  19 - (3 - 1) => 17 * 17 * 32
        # pooling: 8 * 8 * 32
        self.layer2 = nn.Sequential(                # filter = 3 * 3 * 32
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Linear(in_features=8 * 8 * 32, out_features=5749)

    def forward(self,x):
        l1 = self.layer1(x)
        l2 = self.layer2(l1)
        l2 = l2.reshape(l2.size(0), -1)
        l3 = self.fc(l2)
        return l3


#GPU OR CPU
model = ConvNet().to(device=device)

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.01)

model.train()
for i in range(num_epoch):
    for imge, labl in train_dataset:
        #GPU OR CPU
        imge = imge.to(device)
        labl = labl.to(device)

        out = model(imge)
        loss = loss_fn(out, labl)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print("loss : %.4f" % (loss.item()))

loss : 16.3060
loss : 0.4808
loss : 0.0744
loss : 0.0270
loss : 0.0117


In [6]:
from torch.utils.data import random_split, DataLoader

train_dataset = torchvision.datasets.LFWPeople(root='./dataset/',
                                           split="train",
                                           transform=transforms.Compose([
                                               transforms.Resize((28,28)),
                                               transforms.Grayscale(),
                                               transforms.ToTensor()]),
                                           download=True)

number_of_test = int(len(train_dataset)*0.3)
number_of_train = len(train_dataset) - int(len(train_dataset)*0.3)
train_attack_cat ,test_attack_cat = random_split(dataset=train_dataset, lengths=[number_of_train,number_of_test])

test = DataLoader(dataset=test_attack_cat ,batch_size=batch_size ,shuffle=True)

model.eval()
correct = 0
wrong = 0
for i, (imge, labl) in enumerate(test):
  for i1, (imge1, labl1) in enumerate(test):
    #GPU OR CPU
    imge = imge.to(device)
    labl = labl.to(device)
    
    # -------------------
    out = model(imge)
    
    labl1 = labl1.to(device)

    _,predicted = torch.max(out,1)
    
    try:

        correct += torch.sum(predicted == labl1) 
        print("correct %s" % correct)

        wrong += torch.sum(predicted != labl1) 
        print("wrong %s" % wrong)

    except:
      pass

Files already downloaded and verified
correct tensor(8, device='cuda:0')
wrong tensor(992, device='cuda:0')
correct tensor(15, device='cuda:0')
wrong tensor(1985, device='cuda:0')
correct tensor(20, device='cuda:0')
wrong tensor(2980, device='cuda:0')
correct tensor(21, device='cuda:0')
wrong tensor(3979, device='cuda:0')
correct tensor(24, device='cuda:0')
wrong tensor(4833, device='cuda:0')
